# Задание 4

Найдите ТОП-10 организаций, которые за период 2019-2022 заплатили больше всего налогов и при этом платили налог все 4 года: 
* в выгрузке должен быть идентификатор, наименование, адрес, налоги по годам и суммарный налог. 
* сортировка по убыванию уплаты налогов и по названию в алфавитном порядке (для равных по уплате налога).


## Описание датасетов

#### main.csv - *основная информация о предприятиях*
- ID - уникальный номер объекта;
- Name - название;
- Address - адрес

#### net_property.csv - *указание на сетевые предприятия*
- ID - уникальный номер объекта;
- IsNetObject - метка сетевого предприятия;

#### seats_property.csv - *количество посадочных мест*
- ID - уникальный номер объекта;
- SeatsCount - количество посадочных мест;
- ver_ID - номер версии записи

#### type_property.csv - *информация о специализации: столовая, закусочная, кафе, иное*
- ID - уникальный номер объекта;
- TypeObject - тип объекта;
- is_actual - актуальность записи

#### indicators_db.sqlite3 - *SQL-база с данными о выручке предприятий и числе сотрудников*
- данные за 2019 г.;
- данные за 2020-2022 гг.

In [1]:
import pandas as pd
import numpy as np

### Чтение датасетов

In [2]:
df1 = pd.read_csv('main.csv')
df2 = pd.read_csv('net_property.csv')
df3 = pd.read_csv('seats_property.csv')
df4 = pd.read_csv('type_property.csv')

In [3]:
import sqlalchemy as sql

In [4]:
con = sql.create_engine('sqlite:///indicators_db.sqlite3')

In [5]:
df = pd.read_sql("SELECT * FROM sqlite_master WHERE type='table';", con)

In [6]:
df

,type,name,tbl_name,rootpage,sql
0,table,org_indicators_2019,org_indicators_2019,2,CREATE TABLE org_indicators_2019 (\r\n\tID BIG...
1,table,org_indicators_2020_2022,org_indicators_2020_2022,490,CREATE TABLE org_indicators_2020_2022 (\r\n\tI...


In [7]:
df_sql_select_19 = pd.read_sql('select * from org_indicators_2019', con)

In [8]:
df_sql_select_20_22 = pd.read_sql('select * from org_indicators_2020_2022', con)

### Выборка необходимых колонок и разворот через pivot

In [9]:
df_19 = df_sql_select_19[df_sql_select_19['INDICATOR_NAME']  == 'Налоги']

In [10]:
df_19_piv = df_19.pivot(index = 'ID', columns = ['YEAR','INDICATOR_NAME'], values = 'VALUE')

In [11]:
df_20_22 = df_sql_select_20_22[df_sql_select_20_22['INDICATOR_NAME']  == 'Налоги']

In [12]:
df_20_22_piv = df_20_22.pivot(index = 'ID', columns = ['YEAR','INDICATOR_NAME'], values = 'VALUE')

In [13]:
df_20_22_piv

YEAR,2020,2021,2022
INDICATOR_NAME,Налоги,Налоги,Налоги
ID,,,
19630,552636.0,741822.0,612311.0
19643,NaN,NaN,482734.0
19647,616861.0,770743.0,NaN
19657,NaN,NaN,649863.0
19677,500494.0,NaN,NaN
...,...,...,...
375848,769347.0,429547.0,NaN
375884,NaN,NaN,391101.0


### Удаление уровня "YEAR" и переименование колонок

In [14]:
testing_1 = df_19_piv.droplevel([0], axis = 1)
testing_1.columns.names = [None]
testing_1.to_csv('test_19.csv')
testing_1 = pd.read_csv('test_19.csv')
testing_1 = testing_1.rename(columns = {'Налоги':'Налоги (2019 г.)'})

In [15]:
testing_2 = df_20_22_piv.droplevel([0], axis = 1)
testing_2.columns.names = [None]
testing_2.to_csv('test_20.csv')
testing_2 = pd.read_csv('test_20.csv')
rename = {'Налоги':'Налоги (2020 г.)', 'Налоги.1':'Налоги (2021 г.)', 'Налоги.2':'Налоги (2022 г.)'}
testing_2 = testing_2.rename(columns = rename)

In [16]:
testing = testing_1.merge(testing_2, how = 'outer')

In [17]:
testing_fill = testing.fillna(0)

In [18]:
testing_fill['Налоги (Всего)'] = testing_fill['Налоги (2019 г.)'] + testing_fill['Налоги (2020 г.)'] + \
                                 testing_fill['Налоги (2021 г.)'] + testing_fill['Налоги (2022 г.)']

In [19]:
testing_fill

,ID,Налоги (2019 г.),Налоги (2020 г.),Налоги (2021 г.),Налоги (2022 г.),Налоги (Всего)
0,19657,659466.0,0.0,0.0,649863.0,1309329.0
1,19677,359672.0,500494.0,0.0,0.0,860166.0
2,19681,109791.0,0.0,0.0,227324.0,337115.0
3,19688,332925.0,413374.0,794885.0,528160.0,2069344.0
4,19708,473159.0,0.0,205018.0,0.0,678177.0
...,...,...,...,...,...,...
9894,375654,0.0,536539.0,285265.0,0.0,821804.0
9895,375834,0.0,381183.0,0.0,0.0,381183.0
9896,375954,0.0,286080.0,647285.0,685297.0,1618662.0
9897,375956,0.0,122959.0,0.0,0.0,122959.0


In [20]:
df1_test = df1.merge(testing_fill, on = 'ID', how = 'inner')

In [21]:
df_test_sort = df1_test.sort_values(by = ['Налоги (Всего)','Name'], ascending = [False,False]).head(10)

In [22]:
df_test_sort

,ID,Name,Address,Налоги (2019 г.),Налоги (2020 г.),Налоги (2021 г.),Налоги (2022 г.),Налоги (Всего)
9075,259110,Кафе Плов Хаус,"Российская Федерация, город Москва, внутригоро...",781647.0,778240.0,787289.0,690317.0,3037493.0
4530,360850,"Пекарня, шашлыки","Российская Федерация, город Москва, внутригоро...",743416.0,708866.0,782963.0,719909.0,2955154.0
8454,238923,Мята,"Российская Федерация, город Москва, внутригоро...",740341.0,762886.0,666885.0,760670.0,2930782.0
7508,293423,Буханка,"город Москва, улица Полины Осипенко, дом 10, к...",728116.0,758394.0,690844.0,734469.0,2911823.0
4160,205809,SeDelice Французская пекарня,"Российская Федерация, город Москва, внутригоро...",790908.0,728516.0,632571.0,732026.0,2884021.0
771,326049,Кебаб Хаус,"Российская Федерация, город Москва, внутригоро...",773919.0,692267.0,746153.0,652210.0,2864549.0
7409,348665,2Bro,"Российская Федерация, город Москва, внутригоро...",792857.0,531143.0,790359.0,682982.0,2797341.0
2051,288923,Кальян-бар Otrada Lounge,"Российская Федерация, город Москва, внутригоро...",538933.0,689504.0,798771.0,755940.0,2783148.0
1128,303983,Жалал Абад,"Российская Федерация, город Москва, внутригоро...",783555.0,618359.0,680750.0,680120.0,2762784.0
5326,298149,Munterra,"город Москва, Большой Харитоньевский переулок,...",498451.0,758751.0,710255.0,780775.0,2748232.0


In [23]:
renaming = {'ID':'Идентификатор','Name':'Наименование','Address':'Адрес'}
df_test_sort = df_test_sort.rename(columns = renaming)

In [24]:
df_test_sort

,Идентификатор,Наименование,Адрес,Налоги (2019 г.),Налоги (2020 г.),Налоги (2021 г.),Налоги (2022 г.),Налоги (Всего)
9075,259110,Кафе Плов Хаус,"Российская Федерация, город Москва, внутригоро...",781647.0,778240.0,787289.0,690317.0,3037493.0
4530,360850,"Пекарня, шашлыки","Российская Федерация, город Москва, внутригоро...",743416.0,708866.0,782963.0,719909.0,2955154.0
8454,238923,Мята,"Российская Федерация, город Москва, внутригоро...",740341.0,762886.0,666885.0,760670.0,2930782.0
7508,293423,Буханка,"город Москва, улица Полины Осипенко, дом 10, к...",728116.0,758394.0,690844.0,734469.0,2911823.0
4160,205809,SeDelice Французская пекарня,"Российская Федерация, город Москва, внутригоро...",790908.0,728516.0,632571.0,732026.0,2884021.0
771,326049,Кебаб Хаус,"Российская Федерация, город Москва, внутригоро...",773919.0,692267.0,746153.0,652210.0,2864549.0
7409,348665,2Bro,"Российская Федерация, город Москва, внутригоро...",792857.0,531143.0,790359.0,682982.0,2797341.0
2051,288923,Кальян-бар Otrada Lounge,"Российская Федерация, город Москва, внутригоро...",538933.0,689504.0,798771.0,755940.0,2783148.0
1128,303983,Жалал Абад,"Российская Федерация, город Москва, внутригоро...",783555.0,618359.0,680750.0,680120.0,2762784.0
5326,298149,Munterra,"город Москва, Большой Харитоньевский переулок,...",498451.0,758751.0,710255.0,780775.0,2748232.0


In [25]:
df_test_sort.to_csv('result.csv', index=False)